In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch

In [15]:
df = pd.read_csv('https://raw.githubusercontent.com/krishnaik06/Keras-Tuner/main/Real_Combine.csv' , )
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [17]:
X = df.iloc[: , :-1]
y = df.iloc[: , -1]

## Hyperparameters
1. **Number of Hidden Layers**
2. **Number of Neurons in Hidden Layers**
3. **Learning Rate**

## keras.Sequential()

**Here's a simple analogy: think of the sequential model as an empty container (your neural network) that you fill with layers one by one, and the data you pass through the model flows through these layers in a sequential fashion, from the input layer to the output layer.**


**Each neuron in a dense layer is connected to every neuron in the previous layer, making it a fully connected layer.**

In [18]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)): # 2-20 hidden layer
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear')) # activation='linear' because to solve the Regression problem
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

# The hp.Int('num_layers', 2, 20) is part of the Keras Tuner library. It is not a built-in Python function or method.
#It's used to define an integer hyperparameter named 'num_layers' that takes on integer values between 2 and 20

#for each layer, the number of neurons will be a tunable hyperparameter that can take values between 32 and 512 in increments of 32, allowing for a range of possible values within that specified range.

In [19]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project',
    project_name='Air Quality Index')

INFO:tensorflow:Reloading Tuner from project\Air Quality Index\tuner0.json


In [20]:
tuner.search_space_summary()

Search space summary
Default search space size: 14
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_4 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_5 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_6

### max_trials:: 
is the maximum number of different hyperparameter combinations that will be tried during the hyperparameter tuning process. In this case, it's set to 5, which means that the tuner will explore up to 5 different sets of hyperparameters.

### executions_per_trial:: 
is the number of times each set of hyperparameters will be evaluated or trained. In this case, it's set to 3, which means that for each set of hyperparameters, the model will be trained and evaluated 3 times. This is often done to account for variability in model performance due to randomness (e.g., random weight initialization).

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [22]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))


Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
13                |9                 |num_layers
224               |320               |units_0
480               |224               |units_1
0.0001            |0.01              |learning_rate
32                |128               |units_2
384               |128               |units_3
96                |32                |units_4
480               |96                |units_5
192               |224               |units_6
416               |32                |units_7
480               |288               |units_8
160               |512               |units_9
384               |480               |units_10
288               |64                |units_11
32                |None              |units_12

Epoch 1/5
24/24 [==============================] - 9s 43ms/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 2/5
24/24 [==============================] - 0s 16ms/step - l

C:\Users\arups\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras_tuner\engine\metrics_tracking.py:110: RuntimeWarning: All-NaN axis encountered
  np.nanmin(values) if self.direction == "min" else np.nanmax(values)


TypeError: can only concatenate str (not "NoneType") to str

In [23]:
tuner.results_summary()

Results summary
Results in project\Air Quality Index
Showing 10 best trials
Objective(name="val_mean_absolute_error", direction="min")

Trial 1 summary
Hyperparameters:
num_layers: 9
units_0: 320
units_1: 224
learning_rate: 0.01
units_2: 128
units_3: 128
units_4: 32
units_5: 96
units_6: 224
units_7: 32
units_8: 288
units_9: 512
units_10: 480
units_11: 64
Score: nan

Trial 2 summary
Hyperparameters:
num_layers: 13
units_0: 224
units_1: 480
learning_rate: 0.0001
units_2: 32
units_3: 384
units_4: 96
units_5: 480
units_6: 192
units_7: 416
units_8: 480
units_9: 160
units_10: 384
units_11: 288
units_12: 32
Score: nan

Trial 0 summary
Hyperparameters:
num_layers: 12
units_0: 192
units_1: 480
learning_rate: 0.001
units_2: 32
units_3: 32
units_4: 32
units_5: 32
units_6: 32
units_7: 32
units_8: 32
units_9: 32
units_10: 32
units_11: 32
Score: nan
